In [1]:
import torch


from torch_geometric.nn import GCNConv
from torch_geometric.nn.pool import global_mean_pool
from torch.utils.data import DataLoader, random_split

In [2]:
from helpers import CVFConfigForGCNDataset

In [3]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [4]:
class VanillaGNNLayer(torch.nn.Module):
    def __init__(self, dim_in, dim_out):
        super().__init__()
        self.linear = torch.nn.Linear(dim_in, dim_out, bias=False)

    def forward(self, x, adjacency):
        x = self.linear(x)
        # x = torch.sparse.mm(adjacency, x)
        x = torch.matmul(adjacency, x)
        return x

In [5]:
# dataset = CVFConfigDataset(
#     "small_graph_test_config_rank_dataset.csv", "small_graph_edge_index.json", 4
# )
# dataset = CVFConfigDataset(
#     "graph_1_config_rank_dataset.csv", "graph_1_edge_index.json", 10
# )
# dataset = CVFConfigDataset(
#     "graph_4_config_rank_dataset.csv", "graph_4_edge_index.json", 10
# )
# dataset = CVFConfigDataset(
#     "graph_5_config_rank_dataset.csv", "graph_5_edge_index.json", 9
# )
dataset_pl_n8 = CVFConfigForGCNDataset(
    device,
    "graph_6b_config_rank_dataset.csv",
    "graph_6b_edge_index.json",
    20,
)
dataset_pl_n9 = CVFConfigForGCNDataset(
    device,
    "graph_powerlaw_cluster_graph_n9_config_rank_dataset.csv",
    "graph_powerlaw_cluster_graph_n9_edge_index.json",
    20,
)
# dataset = CVFConfigDataset(
#     "graph_powerlaw_cluster_graph_n12_config_rank_dataset.csv",
#     "graph_powerlaw_cluster_graph_n12_edge_index.json",
#     10
# )

batch_size = 128

train_size = int(0.75 * len(dataset_pl_n9))
test_size = len(dataset_pl_n9) - train_size
train_dataset_pl_n9, test_dataset_pl_n9 = random_split(dataset_pl_n9, [train_size, test_size])

train_size = int(0.75 * len(dataset_pl_n8))
test_size = len(dataset_pl_n8) - train_size
train_dataset_pl_n8, test_dataset_pl_n8 = random_split(dataset_pl_n8, [train_size, test_size])

train_loader_pl_n9 = DataLoader(train_dataset_pl_n9, batch_size=batch_size, shuffle=True)
test_loader_pl_n9 = DataLoader(test_dataset_pl_n9, batch_size=batch_size, shuffle=False)

train_loader_pl_n8 = DataLoader(train_dataset_pl_n8, batch_size=batch_size, shuffle=True)
test_loader_pl_n8 = DataLoader(test_dataset_pl_n8, batch_size=batch_size, shuffle=False)

dataset_coll = [dataset_pl_n8, dataset_pl_n9]
dataloader_coll = [train_loader_pl_n8, train_loader_pl_n9]
dataloader_coll_iter = [iter(i) for i in dataloader_coll]
# data_loader = DataLoader(dataset, batch_size=128, shuffle=False)

In [6]:
def generate_batch():
    end_loop = [False for _ in range(len(dataloader_coll))]
    while not all(end_loop):
        for di, data_loader in enumerate(dataloader_coll_iter):
            try:
                batch = next(data_loader)
            except StopIteration:
                end_loop[di] = True
                continue
            yield batch, di

In [7]:
# cnt = 0
# for i in generate_batch():
#     cnt += 1

# print(cnt)
print([len(i) for i in dataloader_coll])

[64, 8100]


In [ ]:
class VanillaGNN(torch.nn.Module):
    def __init__(self, dim_in, dim_h, dim_out):
        super().__init__()
        self.gcn1 = GCNConv(dim_in, dim_h)
        self.gcn2 = GCNConv(dim_h, dim_h)
        self.out = torch.nn.Linear(dim_h, dim_out)

    def forward(self, x, edge_index):
        h = self.gcn1(x, edge_index)
        h = torch.relu(h)
        h = self.gcn2(h, edge_index)
        h = torch.relu(h)
        h = self.out(h)
        h = global_mean_pool(h, torch.zeros(h.size(1)).to(device).long())
        return h

    def fit(self, epochs):
        # criterion = torch.nn.CrossEntrop yLoss()
        criterion = torch.nn.MSELoss()
        optimizer = torch.optim.Adam(self.parameters(), lr=0.01, weight_decay=5e-4)
        self.train()
        # edge_index = dataset.edge_index.t().to(device)
        for epoch in range(1, epochs + 1):
            avg_loss = 0
            count = 0
            for batch, di in generate_batch():
                x = batch[0].to(device)
                y = batch[1].to(device)
                y = y.unsqueeze(0).reshape(-1, 1, 1).float()
                optimizer.zero_grad()
                out = self(x, dataset_coll[di].edge_index)
                # print("output", out, "y", y)
                loss = criterion(out, y)
                avg_loss += loss
                count += 1
                loss.backward()
                optimizer.step()
                # if i % 10 == 0:
                #     print("Iter", i, "| Loss:", avg_loss / count)

            if count > 0:
                print("Epoch", epoch, "| Loss:", avg_loss / count)


In [9]:
gnn = VanillaGNN(20, 128, 1).to(device)
print(gnn)

gnn.fit(epochs=10)

VanillaGNN(
  (gcn1): GCNConv(20, 128)
  (gcn2): GCNConv(128, 128)
  (out): Linear(in_features=128, out_features=1, bias=True)
)
Epoch 1 | Loss: tensor(0.4299, device='cuda:0', grad_fn=<DivBackward0>)


ZeroDivisionError: division by zero

In [ ]:
# testing
torch.no_grad()
torch.set_printoptions(profile="full")

total_matched = 0
edge_index = dataset.edge_index.t().to(device)

for batch in test_loader:
    x = batch[0].to(device)
    y = batch[1].to(device)
    y = y.unsqueeze(0).reshape(-1, 1, 1).float()
    predicted = gnn(x, edge_index)
    # predicted = predicted.argmax(dim=1)
    predicted = torch.round(predicted)
    matched = (predicted == y).sum().item()
    total_matched += matched
    

print("Total matched", total_matched, "out of", len(test_dataset), "| Accuracy", round(total_matched/len(test_dataset) * 100, 4), "%")